In [9]:
import sys
from itertools import combinations

import networkx as nx

from utils import get_distance, get_distance_table


# def dfs(graph, start, visited, parent):
#     visited[start] = True
#     for v in graph[start]:
#         if not visited[v]:
#             parent[v] = start
#             dfs(graph, v, visited, parent)
#
#
# def generate_all_spanning_trees(n):
#     graph = {i: set(range(n)) - {i} for i in range(n)}
#     trees = []
#     for edges in combinations(combinations(range(n), 2), n - 1):
#         graph_copy = {i: set() for i in range(n)}
#         for edge in edges:
#             graph_copy[edge[0]].add(edge[1])
#             graph_copy[edge[1]].add(edge[0])
#         visited = [False] * n
#         parent = [-1] * n
#         dfs(graph_copy, 0, visited, parent)
#         if all(visited):
#             trees.append(parent)
#     return trees
#
#
# # Example usage:
# n = 4
# trees = generate_all_spanning_trees(n)
# for tree in trees:
#     print(tree)

In [10]:
# def generate_edges(tree):
#     edges = []
#     for i in range(1, len(tree)):
#         edges.append((i, tree[i]))
#     return edges

In [11]:
# n = 4
# trees = generate_all_spanning_trees(n)
#
# for tree in trees:
#     edges = generate_edges(tree)
#     print(edges)

In [12]:
# def generate_spanning_trees(n):
#     edges = [(i, j) for i in range(n) for j in range(i+1, n)]
#     for i in range(2**(n*(n-1)//2)):
#         tree_edges = []
#         idx = i
#         for u, v in edges:
#             if idx % 2 == 1:
#                 tree_edges.append((u, v))
#             idx //= 2
#         if is_spanning_tree(n, tree_edges):
#             yield tree_edges
#
# def is_spanning_tree(n, edges):
#     if len(edges) != n-1:
#         return False
#     uf = UnionFind(n)
#     for u, v in edges:
#         if uf.find(u) == uf.find(v):
#             return False
#         uf.union(u, v)
#     return True
#
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1


def generate_spanning_trees(n):
    edges = [(i, j) for i in range(n) for j in range(i + 1, n)]
    for i in range(2 ** (n * (n - 1) // 2)):
        tree_edges = []
        idx = i
        for u, v in edges:
            if idx % 2 == 1:
                tree_edges.append((u, v))
            idx //= 2
        if is_spanning_tree(n, tree_edges):
            yield tree_edges


def is_spanning_tree(n, edges):
    if len(edges) != n - 1:
        return False
    uf = UnionFind(n)
    for u, v in edges:
        if uf.find(u) == uf.find(v):
            return False
        uf.union(u, v)
    return True

In [13]:
# G = nx.from_edgelist([(0, 1), (0, 2), (0, 3), (0, 4)])
# print(G.degree())

In [14]:
def calculate_fitness(list_edges, degree_constrained, distances_table):
    G = nx.from_edgelist(list_edges)
    degrees = list(map(lambda x: x[1], G.degree()))
    # print(degrees)

    # Check if the degrees more than target degrees
    if any(x > degree_constrained for x in degrees):
        return sys.maxsize

    cost = 0
    for edge in list_edges:
        cost = cost + get_distance(edge, distances_table)
    return cost
# dis_tab = get_distance_table("data/7_wi29.csv")
# print(calculate_fitness(G.edges(), 4, dis_tab))

In [15]:
import math
import csv


def get_distance_table(file_path):
    records = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # next(reader) # Skip header row
        for row in reader:
            records.append(row)

    # Tạo từ điển để lưu trữ khoảng cách giữa các điểm
    distances_table = {}

    # Tính toán khoảng cách giữa các điểm và lưu trữ kết quả vào từ điển
    for i in range(len(records)):
        for j in range(i + 1, len(records)):
            point1 = (float(records[i][1]), float(records[i][2]))
            point2 = (float(records[j][1]), float(records[j][2]))
            distance = math.dist(point1, point2)
            key = (int(records[i][0]), int(records[j][0]))
            distances_table[key] = distance

    # In ra từ điển khoảng cách
    return distances_table

In [16]:

degree_constrained = 2
dis_tab = get_distance_table("data/5_wi29.csv")
print(dis_tab)
n = 5
spanning_trees = list(generate_spanning_trees(n))
# i = 1
min_cost = sys.maxsize
best_tree = []
for tree in spanning_trees:
    cost = calculate_fitness(tree, degree_constrained=degree_constrained, distances_table=dis_tab)
    # print(cost, tree)
    if cost < min_cost:
        best_tree = tree
        min_cost = cost
    # print("best_tree", min_cost, best_tree)

    # i += 1
print("Tree:", best_tree )
print("Cost: ", min_cost)

{(0, 1): 74.53561415712696, (0, 2): 4109.913459889123, (0, 3): 3047.9957068357057, (0, 4): 2266.911731360042, (1, 2): 4069.7051490249282, (1, 3): 2999.490729922148, (1, 4): 2213.594362117867, (2, 3): 1172.3669941144242, (2, 4): 1972.9419656948855, (3, 4): 816.6666999999998}
Tree: [(0, 1), (1, 4), (2, 3), (3, 4)]
Cost:  4277.163670389418


In [17]:

degree_constrained = 2
dis_tab = get_distance_table("data/7_wi29.csv")
print(dis_tab)
n = 7
spanning_trees = list(generate_spanning_trees(n))
# i = 1
min_cost = sys.maxsize
best_tree = []
for tree in spanning_trees:
    cost = calculate_fitness(tree, degree_constrained=degree_constrained, distances_table=dis_tab)
    # print(cost, tree)
    if cost < min_cost:
        best_tree = tree
        min_cost = cost
    # print("best_tree", min_cost, best_tree)

    # i += 1
print("Tree:", best_tree )
print("Cost: ", min_cost)

{(0, 1): 74.53561415712696, (0, 2): 4109.913459889123, (0, 3): 3047.9957068357057, (0, 4): 2266.911731360042, (0, 5): 973.5388173508504, (0, 6): 4190.100799370928, (1, 2): 4069.7051490249282, (1, 3): 2999.490729922148, (1, 4): 2213.594362117867, (1, 5): 900.6170933803621, (1, 6): 4137.397248808054, (2, 3): 1172.3669941144242, (2, 4): 1972.9419656948855, (2, 5): 3496.2280930867323, (2, 6): 891.0043851993363, (3, 4): 816.6666999999998, (3, 5): 2350.0, (3, 6): 1172.1300771577262, (4, 5): 1533.3333000000002, (4, 6): 1923.899450190796, (5, 6): 3416.829291576882}
Tree: [(0, 1), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5)]
Cost:  5388.287169894551
